***GENERATED CODE FOR eligdenresolutionrecommendationautoml PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = transformationDF.drop('DateOfService')
        transformationDF = transformationDF.drop('TransactionDate')
        transformationDF = transformationDF.drop('ReasonDate')
        transformationDF = transformationDF.drop('BillDate')
        transformationDF = transformationDF.drop('DateOfEntry')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '231471.33', 'stddev': '1033780.35', 'min': '00140', 'max': 'V2785', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '12.54', 'stddev': '20.15', 'min': '  ', 'max': 'TC', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier'}, {'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier2', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '1.38', 'stddev': '8.01', 'min': '  ', 'max': 'RT', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier2'}, {'feature_label': 'CptModifier2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '351.69', 'stddev': '105.27', 'min': '224.5', 'max': 'Z98.890', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '369.8', 'stddev': '6.1', 'min': '362.02', 'max': 'Z98.890', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'DepartmentCategory13', 'max': 'DepartmentCategory9', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'SpecialityCategory-11_Demo3', 'max': 'SpecialityCategory6465_Demo3', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SpecialityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'Aetna Commercial', 'max': 'Workmans Comp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerCategory'}, {'feature_label': 'PrimaryPayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'Aetna Commercial', 'max': 'Workers Comp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerGroup'}, {'feature_label': 'PrimaryPayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerName', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'AARP', 'max': 'Zenith American Solutions', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerName'}, {'feature_label': 'PrimaryPayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerSubGroup1', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerSubGroup1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'NOT AVAILABLE', 'max': 'NOT AVAILABLE', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerSubGroup1'}, {'feature_label': 'PrimaryPayerSubGroup1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerSubGroup1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerSubGroup2', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerSubGroup2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'NOT AVAILABLE', 'max': 'NOT AVAILABLE', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerSubGroup2'}, {'feature_label': 'PrimaryPayerSubGroup2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerSubGroup2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'Aetna Commercial', 'max': 'Workmans Comp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerCategory'}, {'feature_label': 'SecondPayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'Aetna Commercial', 'max': 'Workers Comp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerGroup'}, {'feature_label': 'SecondPayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerName', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'AARP', 'max': 'Zenith American Solutions', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerName'}, {'feature_label': 'SecondPayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerSubGroup1', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerSubGroup1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'NOT AVAILABLE', 'max': 'NOT AVAILABLE', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerSubGroup1'}, {'feature_label': 'SecondPayerSubGroup1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerSubGroup1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerSubGroup2', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerSubGroup2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'NOT AVAILABLE', 'max': 'NOT AVAILABLE', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerSubGroup2'}, {'feature_label': 'SecondPayerSubGroup2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerSubGroup2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TeritoryPayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'TeritoryPayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'Aetna Commercial', 'max': 'United Commercial', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TeritoryPayerCategory'}, {'feature_label': 'TeritoryPayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TeritoryPayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TeritoryPayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'TeritoryPayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'Aetna Commercial', 'max': 'United Medicare', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TeritoryPayerGroup'}, {'feature_label': 'TeritoryPayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TeritoryPayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TeritoryPayerName', 'transformation_label': 'String Indexer'}], 'feature': 'TeritoryPayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'AARP Healthcare Supplement  36273', 'max': 'Yutiq Copay Program', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TeritoryPayerName'}, {'feature_label': 'TeritoryPayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TeritoryPayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TeritoryPayerSubGroup1', 'transformation_label': 'String Indexer'}], 'feature': 'TeritoryPayerSubGroup1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'NOT AVAILABLE', 'max': 'NOT AVAILABLE', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TeritoryPayerSubGroup1'}, {'feature_label': 'TeritoryPayerSubGroup1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TeritoryPayerSubGroup1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TeritoryPayerSubGroup2', 'transformation_label': 'String Indexer'}], 'feature': 'TeritoryPayerSubGroup2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'NOT AVAILABLE', 'max': 'NOT AVAILABLE', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TeritoryPayerSubGroup2'}, {'feature_label': 'TeritoryPayerSubGroup2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TeritoryPayerSubGroup2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ServiceItemName', 'transformation_label': 'String Indexer'}], 'feature': 'ServiceItemName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '235345.46', 'stddev': '1045269.11', 'min': '00140', 'max': 'V2785', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ServiceItemName'}, {'feature_label': 'ServiceItemName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ServiceItemName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ServiceItemCategory', 'transformation_label': 'String Indexer'}], 'feature': 'ServiceItemCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'Anes- Eye; Corneal Transpl', 'max': 'Vitrectomy, Mechanical, Pars Plana Approach', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ServiceItemCategory'}, {'feature_label': 'ServiceItemCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ServiceItemCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ServiceLocation', 'transformation_label': 'String Indexer'}], 'feature': 'ServiceLocation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'Ambulatory Surgical Center', 'max': 'On Campus-Outpatient Hospital', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ServiceLocation'}, {'feature_label': 'ServiceLocation', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ServiceLocation')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Reasoncode', 'transformation_label': 'String Indexer'}], 'feature': 'Reasoncode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '85.83', 'stddev': '80.85', 'min': '107', 'max': 'P14', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Reasoncode'}, {'feature_label': 'Reasoncode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Reasoncode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'Authorization', 'max': 'Requires Review', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonCategory'}, {'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonDescription', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'A facility is responsible for payment to outside providers who furnish these services/supplies/drugs to its patients/residents.', 'max': '`New Patient` qualifications were not met.', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonDescription'}, {'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonDescription')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'TaskCompletedDate', 'transformation_label': 'Extract Date'}], 'feature': 'TaskCompletedDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'TaskCompletedDate'}, {'feature_label': 'TaskCompletedDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('TaskCompletedDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'TaskCreatedDate', 'transformation_label': 'Extract Date'}], 'feature': 'TaskCreatedDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'TaskCreatedDate'}, {'feature_label': 'TaskCreatedDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('TaskCreatedDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TaskCompletionReason', 'transformation_label': 'String Indexer'}], 'feature': 'TaskCompletionReason', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'COB Not Update-Released To Pt', 'max': 'Refiled Claim', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TaskCompletionReason'}, {'feature_label': 'TaskCompletionReason', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TaskCompletionReason')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TaskCompletionReasonCategory', 'transformation_label': 'String Indexer'}], 'feature': 'TaskCompletionReasonCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'COB Not Update-Released To Pt', 'max': 'Refiled Claim', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TaskCompletionReasonCateg...'}, {'feature_label': 'TaskCompletionReasonCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'TaskCompletionReasonCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TaskType', 'transformation_label': 'String Indexer'}], 'feature': 'TaskType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': '90 Days No Response', 'max': 'Waystar Eligibility Error', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TaskType'}, {'feature_label': 'TaskType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TaskType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TaskTypeCategory', 'transformation_label': 'String Indexer'}], 'feature': 'TaskTypeCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'AR', 'max': 'Unknown', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TaskTypeCategory'}, {'feature_label': 'TaskTypeCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TaskTypeCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TaskTypeGroup', 'transformation_label': 'String Indexer'}], 'feature': 'TaskTypeGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'RCM', 'max': 'Unknown', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TaskTypeGroup'}, {'feature_label': 'TaskTypeGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TaskTypeGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TaskStatus', 'transformation_label': 'String Indexer'}], 'feature': 'TaskStatus', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'Completed', 'max': 'Completed', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TaskStatus'}, {'feature_label': 'TaskStatus', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TaskStatus')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TaskStatusCategory', 'transformation_label': 'String Indexer'}], 'feature': 'TaskStatusCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '', 'stddev': '', 'min': 'Completed', 'max': 'Completed', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TaskStatusCategory'}, {'feature_label': 'TaskStatusCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TaskStatusCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'label_resolution', 'transformation_label': 'String Indexer'}], 'feature': 'label_resolution', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3984', 'mean': '11.47', 'stddev': '8.2', 'min': '0', 'max': '5', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'label_resolution'}, {'feature_label': 'label_resolution', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('label_resolution')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run eligdenresolutionrecommendationautomlHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'WorkflowAnalyticsDev', 'optionalDB': 'WorkflowAnalyticsDev', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT ChargeDetailID, DateOfService, TransactionDate,\r\n\tReasonDate,BillDate,DateOfEntry,\r\n\tCPTCode,CptModifier, CptModifier2,\r\n\tpdiagnosis1,pdiagnosis2,\r\n\tAge,Gender,Amount,DepartmentCategory, SpecialityCategory,PrimaryPayerCategory,PrimaryPayerGroup,PrimaryPayerName,PrimaryPayerSubGroup1, PrimaryPayerSubGroup2,\r\n\tSecondPayerCategory, SecondPayerGroup, SecondPayerName, SecondPayerSubGroup1, SecondPayerSubGroup2, TeritoryPayerCategory, TeritoryPayerGroup, TeritoryPayerName, TeritoryPayerSubGroup1,\r\n\tTeritoryPayerSubGroup2, ServiceItemName, ServiceItemCategory, ServiceLocation, \r\n\t\tReasoncode,ReasonCategory,ReasonDescription,\r\n\t\tTaskCompletionReasonID,TaskCompletedDate, TaskCreatedDate,\r\n\t\tTaskCompletionReason,TaskCompletionReasonCategory ,\r\n\t\tTaskType, TaskTypeCategory, TaskTypeGroup,\r\n\t\tTaskStatus, TaskStatusCategory\r\n\t,label_resolution\r\n\r\n\r\nFROM (\r\n\tSELECT ChargeDetailID, DateOfService, TransactionDate,\r\n\tReasonDate,BillDate,DateOfEntry,\r\n\tCPTCode,CptModifier, CptModifier2,\r\n\tpdiagnosis1,pdiagnosis2,\r\n\tAge,Gender,Amount,DepartmentCategory, SpecialityCategory,PrimaryPayerCategory,PrimaryPayerGroup,PrimaryPayerName,PrimaryPayerSubGroup1, PrimaryPayerSubGroup2,\r\n\tSecondPayerCategory, SecondPayerGroup, SecondPayerName, SecondPayerSubGroup1, SecondPayerSubGroup2, TeritoryPayerCategory, TeritoryPayerGroup, TeritoryPayerName, TeritoryPayerSubGroup1,\r\n\tTeritoryPayerSubGroup2, ServiceItemName, ServiceItemCategory, ServiceLocation, \r\n\t\tReasoncode,ReasonCategory,ReasonDescription,\r\n\t\tTaskCompletionReasonID,TaskCompletedDate, TaskCreatedDate,\r\n\t\tTaskCompletionReason,TaskCompletionReasonCategory ,\r\n\t\tTaskType, TaskTypeCategory, TaskTypeGroup,\r\n\t\tTaskStatus, TaskStatusCategory\r\n\t\t,label_resolution\r\n\r\n\t\r\n\t,ROW_NUMBER () OVER (PARTITION BY ChargeDetailID, ReasonDate ORDER BY ReasonDate, TransactionDate) AS RNO\r\n\r\n\tFROM \r\n\t(\r\n\tSELECT DISTINCT\r\n\t\t\t\tcd.ChargeDetailID, \r\n\t\t\t\tcd.DateofService, TransactionDate,tr.ReasonDate,b.billdate,cd.DateOfEntry,\r\n\t\t\t\tcptc.CPTCode,\r\n\t\t\t\tCASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\n\t\t\t\tCASE WHEN cd.CPTModifier2 IS NOT NULL THEN cd.CPTModifier2 ELSE '0000' END as CptModifier2,\r\n\t\t\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\t\t\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\t\t\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\n\t            cd.Amount,\r\n\t\t\t\td.DepartmentCategory,\r\n\t\t\t\ts.SpecialityCategory,\r\n\t\r\n\t\t\t\tCASE WHEN py.PayerCategory IS NOT NULL THEN py.PayerCategory ELSE 'NOT AVAILABLE' END as PrimaryPayerCategory ,\r\n\t\t\t\tCASE WHEN py.PayerGroup IS NOT NULL THEN py.PayerGroup ELSE 'NOT AVAILABLE' END as PrimaryPayerGroup ,\r\n\t\t\t\tCASE WHEN py.PayerName IS NOT NULL THEN py.PayerName ELSE 'NOT AVAILABLE' END as PrimaryPayerName ,\r\n\t\t\t\tCASE WHEN py.PayerSubGroup1 IS NOT NULL THEN py.PayerSubGroup1 ELSE 'NOT AVAILABLE' END as PrimaryPayerSubGroup1 ,\r\n\t\t\t\tCASE WHEN py.PayerSubGroup2 IS NOT NULL THEN py.PayerSubGroup2 ELSE 'NOT AVAILABLE' END as PrimaryPayerSubGroup2 ,\r\n\t\r\n\t\t\t\tCASE WHEN py1.PayerCategory IS NOT NULL THEN py1.PayerCategory ELSE 'NOT AVAILABLE' END as SecondPayerCategory ,\r\n\t\t\t\tCASE WHEN py1.PayerGroup IS NOT NULL THEN py1.PayerGroup ELSE 'NOT AVAILABLE' END as SecondPayerGroup ,\r\n\t\t\t\tCASE WHEN py1.PayerName IS NOT NULL THEN py1.PayerName ELSE 'NOT AVAILABLE' END as SecondPayerName ,\r\n\t\t\t\tCASE WHEN py1.PayerSubGroup1 IS NOT NULL THEN py1.PayerSubGroup1 ELSE 'NOT AVAILABLE' END as SecondPayerSubGroup1 ,\r\n\t\t\t\tCASE WHEN py1.PayerSubGroup2 IS NOT NULL THEN py1.PayerSubGroup2 ELSE 'NOT AVAILABLE' END as SecondPayerSubGroup2 ,\r\n\t\r\n\t\t\t\tCASE WHEN py2.PayerCategory IS NOT NULL THEN py2.PayerCategory ELSE 'NOT AVAILABLE' END as TeritoryPayerCategory ,\r\n\t\t\t\tCASE WHEN py2.PayerGroup IS NOT NULL THEN py2.PayerGroup ELSE 'NOT AVAILABLE' END as TeritoryPayerGroup ,\r\n\t\t\t\tCASE WHEN py2.PayerName IS NOT NULL THEN py2.PayerName ELSE 'NOT AVAILABLE' END as TeritoryPayerName ,\r\n\t\t\t\tCASE WHEN py2.PayerSubGroup1 IS NOT NULL THEN py2.PayerSubGroup1 ELSE 'NOT AVAILABLE' END as TeritoryPayerSubGroup1 ,\r\n\t\t\t\tCASE WHEN py2.PayerSubGroup2 IS NOT NULL THEN py2.PayerSubGroup2 ELSE 'NOT AVAILABLE' END as TeritoryPayerSubGroup2 ,\r\n\t\r\n\t\t\t\tCASE WHEN si.ServiceItemName IS NOT NULL THEN si.ServiceItemName ELSE 'NOT AVAILABLE' END as ServiceItemName ,\r\n\t\t\t\tCASE WHEN si.ServiceItemCategory IS NOT NULL THEN si.ServiceItemCategory ELSE 'NOT AVAILABLE' END as ServiceItemCategory,\r\n\t\t\t\tCASE WHEN sl.ServiceLocation IS NOT NULL THEN sl.ServiceLocation ELSE 'NOT AVAILABLE' END as ServiceLocation,\r\n\r\n\t\t\ttrc.Reasoncode,\r\n\t\t\ttrc.ReasonCategory,\r\n\t\t\ttrc.ReasonDescription,\r\n\t\t\ttsk.TaskCompletionReasonID,tsk.CompletedDate as TaskCompletedDate, tsk.CreatedDate as TaskCreatedDate, \r\n\t\t\ttcr.TaskCompletionReason,tcr.TaskCompletionReasonCategory ,\r\n\t\t\ttskt.TaskType, tskt.TaskTypeCategory, tskt.TaskTypeGroup,\r\n\t\t\tts.TaskStatus, ts.TaskStatusCategory,\r\n\r\n\tCASE \r\n\t\tWHEN tcr.TaskCompletionReasonID = '64' THEN '1'\r\n\t\tWHEN tcr.TaskCompletionReasonID = '56' THEN '2'\r\n\t\tWHEN tcr.TaskCompletionReasonID = '44' THEN '3'\r\n\t\tWHEN tcr.TaskCompletionReasonID = '51' THEN '4'\r\n\t\tWHEN tcr.TaskCompletionReasonID = '48' THEN '5'\r\n\t\tWHEN tcr.TaskCompletionReasonID = '45' THEN '6'\r\n\t\tWHEN tcr.TaskCompletionReasonID = '41' THEN '7'\r\n\t\tWHEN tcr.TaskCompletionReasonID = '57' THEN '8'\r\n\t\tWHEN tcr.TaskCompletionReasonID = '84' THEN '9'\r\n\t\tWHEN tcr.TaskCompletionReasonID = '62' THEN '10'\r\n\t\tWHEN tcr.TaskCompletionReasonID = '43' THEN '11'\r\n\t\tWHEN tcr.TaskCompletionReasonID = '13' THEN '12'\r\n\t\tWHEN tcr.TaskCompletionReasonID = '49' THEN '13'\r\n\t\tWHEN tcr.TaskCompletionReasonID = '63' THEN '14'\r\n\t\tWHEN tcr.TaskCompletionReasonID = '42' THEN '15'\r\n\t\tWHEN tcr.TaskCompletionReasonID = '67' THEN '16'\r\n\t\tWHEN tcr.TaskCompletionReasonID IN ('3','12') THEN '17'\r\n\t\tWHEN tcr.TaskCompletionReasonID IN ('65','61','8','6','16','85','79','80') THEN '18'\r\n\t\tWHEN tcr.TaskCompletionReasonID IN ('14','46','90','4','9','68','73','75','5','7','10','76','50','59','60') THEN '19'\r\n\t\tELSE '0' END AS label_resolution\r\n\t\r\n\tfrom ChargeDetail cd\r\n\tinner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\n\tinner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID  \r\n\tleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerId = t.PayerID \r\n\tinner join Bill b WITH (NOLOCK) on e.EncounterID = b.EncounterID and b.BillID = t.BillID \r\n\tinner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID AND T.TransactionDate>=TR.ReasonDate\r\n\tinner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\n\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\tinner join Payer py WITH (NOLOCK) on py.PayerID = t.PayerID\r\n\tleft join Payer py1 WITH (NOLOCK) on py1.PayerID = cd.SecondaryPayerID\r\n\tleft join Payer py2 WITH (NOLOCK) on py2.PayerID = cd.TertiaryPayerID\r\n\tinner join ServiceItem si WITH (NOLOCK) on cd.ServiceItemID = si.ServiceItemID\r\n\tleft outer join ServiceLocation sl WITH (NOLOCK) on cd.ServiceLocationID = sl.ServiceLocationID\r\n\t\t\t\r\n\t\t\tinner join Transactiontype tt on tt.transactiontypeid = t.transactiontypeid\r\n\t\t\tleft outer join  Task tsk WITH (NOLOCK) on e.encounterid = tsk.encounterid and tsk.PayerId = e.PayerID \r\n\t\t\tleft outer join  TaskCompletionReason tcr WITH (NOLOCK) on tsk.TaskCompletionReasonID = tcr.TaskCompletionReasonID  \r\n\t\t\tleft outer join  Taskstatus ts on tsk.TaskStatusID=ts.TaskStatusID\r\n\t\t\tleft outer join  TaskType tskt on tsk.TaskTypeID=tskt.TaskTypeID\r\n\r\nWHERE\t\tcd.Amount >.01\r\n\t\t\tand trc.reasonType = 'Denial'\r\n\t\t\tand ts.taskstatus = 'Completed'\r\n\t\t\tand TransactionDate >= tsk.CompletedDate and t.transactiontypeid=2 and tr.Amount>=0\r\n\t\t\tand tr.ReasonDate between '2018-01-01' and '2020-12-31'\r\n\t\t\tand ( ( tcr.TaskCompletionReasonID Between 41 and 90) OR (tcr.TaskCompletionReasonID IN ('789','28512','30899'))\r\n\t\t\tOR ( tcr.TaskCompletionReasonID Between 3 and 16) OR (tcr.TaskCompletionReasonID IN ('34271')))\r\n\t) C\r\n\r\n) D\r\nWHERE RNO = 1", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run eligdenresolutionrecommendationautomlHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "0022482A-1C07-4B9A-B657-0B270B2EE4C6", "max": "FFFBC98A-041F-4710-932C-06718BDF7B3C", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{"feature_label": "DateOfService", "transformation_label": "Extract Date"}], "feature": "DateOfService", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "DateOfService"}, {"transformationsData": [{"feature_label": "TransactionDate", "transformation_label": "Extract Date"}], "feature": "TransactionDate", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "TransactionDate"}, {"transformationsData": [{"feature_label": "ReasonDate", "transformation_label": "Extract Date"}], "feature": "ReasonDate", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "ReasonDate"}, {"transformationsData": [{"feature_label": "BillDate", "transformation_label": "Extract Date"}], "feature": "BillDate", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "BillDate"}, {"transformationsData": [{"feature_label": "DateOfEntry", "transformation_label": "Extract Date"}], "feature": "DateOfEntry", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "DateOfEntry"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "231471.33", "stddev": "1033780.35", "min": "00140", "max": "V2785", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"feature_label": "CptModifier", "transformation_label": "String Indexer"}], "feature": "CptModifier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "12.54", "stddev": "20.15", "min": "  ", "max": "TC", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier"}, {"transformationsData": [{"feature_label": "CptModifier2", "transformation_label": "String Indexer"}], "feature": "CptModifier2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "1.38", "stddev": "8.01", "min": "  ", "max": "RT", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier2"}, {"transformationsData": [{"feature_label": "pdiagnosis1", "transformation_label": "String Indexer"}], "feature": "pdiagnosis1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "351.69", "stddev": "105.27", "min": "224.5", "max": "Z98.890", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis1"}, {"transformationsData": [{"feature_label": "pdiagnosis2", "transformation_label": "String Indexer"}], "feature": "pdiagnosis2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "369.8", "stddev": "6.1", "min": "362.02", "max": "Z98.890", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3984", "mean": "68.4", "stddev": "15.34", "min": "3", "max": "104", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3984", "mean": "429.2", "stddev": "815.88", "min": "2.0", "max": "7700.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "DepartmentCategory13", "max": "DepartmentCategory9", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "SpecialityCategory", "transformation_label": "String Indexer"}], "feature": "SpecialityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "SpecialityCategory-11_Demo3", "max": "SpecialityCategory6465_Demo3", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SpecialityCategory"}, {"transformationsData": [{"feature_label": "PrimaryPayerCategory", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "Aetna Commercial", "max": "Workmans Comp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerCategory"}, {"transformationsData": [{"feature_label": "PrimaryPayerGroup", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "Aetna Commercial", "max": "Workers Comp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerGroup"}, {"transformationsData": [{"feature_label": "PrimaryPayerName", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "AARP", "max": "Zenith American Solutions", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerName"}, {"transformationsData": [{"feature_label": "PrimaryPayerSubGroup1", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerSubGroup1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "NOT AVAILABLE", "max": "NOT AVAILABLE", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerSubGroup1"}, {"transformationsData": [{"feature_label": "PrimaryPayerSubGroup2", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerSubGroup2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "NOT AVAILABLE", "max": "NOT AVAILABLE", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerSubGroup2"}, {"transformationsData": [{"feature_label": "SecondPayerCategory", "transformation_label": "String Indexer"}], "feature": "SecondPayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "Aetna Commercial", "max": "Workmans Comp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerCategory"}, {"transformationsData": [{"feature_label": "SecondPayerGroup", "transformation_label": "String Indexer"}], "feature": "SecondPayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "Aetna Commercial", "max": "Workers Comp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerGroup"}, {"transformationsData": [{"feature_label": "SecondPayerName", "transformation_label": "String Indexer"}], "feature": "SecondPayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "AARP", "max": "Zenith American Solutions", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerName"}, {"transformationsData": [{"feature_label": "SecondPayerSubGroup1", "transformation_label": "String Indexer"}], "feature": "SecondPayerSubGroup1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "NOT AVAILABLE", "max": "NOT AVAILABLE", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerSubGroup1"}, {"transformationsData": [{"feature_label": "SecondPayerSubGroup2", "transformation_label": "String Indexer"}], "feature": "SecondPayerSubGroup2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "NOT AVAILABLE", "max": "NOT AVAILABLE", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerSubGroup2"}, {"transformationsData": [{"feature_label": "TeritoryPayerCategory", "transformation_label": "String Indexer"}], "feature": "TeritoryPayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "Aetna Commercial", "max": "United Commercial", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TeritoryPayerCategory"}, {"transformationsData": [{"feature_label": "TeritoryPayerGroup", "transformation_label": "String Indexer"}], "feature": "TeritoryPayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "Aetna Commercial", "max": "United Medicare", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TeritoryPayerGroup"}, {"transformationsData": [{"feature_label": "TeritoryPayerName", "transformation_label": "String Indexer"}], "feature": "TeritoryPayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "AARP Healthcare Supplement  36273", "max": "Yutiq Copay Program", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TeritoryPayerName"}, {"transformationsData": [{"feature_label": "TeritoryPayerSubGroup1", "transformation_label": "String Indexer"}], "feature": "TeritoryPayerSubGroup1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "NOT AVAILABLE", "max": "NOT AVAILABLE", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TeritoryPayerSubGroup1"}, {"transformationsData": [{"feature_label": "TeritoryPayerSubGroup2", "transformation_label": "String Indexer"}], "feature": "TeritoryPayerSubGroup2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "NOT AVAILABLE", "max": "NOT AVAILABLE", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TeritoryPayerSubGroup2"}, {"transformationsData": [{"feature_label": "ServiceItemName", "transformation_label": "String Indexer"}], "feature": "ServiceItemName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "235345.46", "stddev": "1045269.11", "min": "00140", "max": "V2785", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ServiceItemName"}, {"transformationsData": [{"feature_label": "ServiceItemCategory", "transformation_label": "String Indexer"}], "feature": "ServiceItemCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "Anes- Eye; Corneal Transpl", "max": "Vitrectomy, Mechanical, Pars Plana Approach", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ServiceItemCategory"}, {"transformationsData": [{"feature_label": "ServiceLocation", "transformation_label": "String Indexer"}], "feature": "ServiceLocation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "Ambulatory Surgical Center", "max": "On Campus-Outpatient Hospital", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ServiceLocation"}, {"transformationsData": [{"feature_label": "Reasoncode", "transformation_label": "String Indexer"}], "feature": "Reasoncode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "85.83", "stddev": "80.85", "min": "107", "max": "P14", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Reasoncode"}, {"transformationsData": [{"feature_label": "ReasonCategory", "transformation_label": "String Indexer"}], "feature": "ReasonCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "Authorization", "max": "Requires Review", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonCategory"}, {"transformationsData": [{"feature_label": "ReasonDescription", "transformation_label": "String Indexer"}], "feature": "ReasonDescription", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "A facility is responsible for payment to outside providers who furnish these services/supplies/drugs to its patients/residents.", "max": "`New Patient` qualifications were not met.", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonDescription"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TaskCompletionReasonID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3984", "mean": "18.22", "stddev": "14.03", "min": "7", "max": "52", "missing": "0"}, "updatedLabel": "TaskCompletionReasonID"}, {"transformationsData": [{"feature_label": "TaskCompletedDate", "transformation_label": "Extract Date"}], "feature": "TaskCompletedDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "TaskCompletedDate"}, {"transformationsData": [{"feature_label": "TaskCreatedDate", "transformation_label": "Extract Date"}], "feature": "TaskCreatedDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "TaskCreatedDate"}, {"transformationsData": [{"feature_label": "TaskCompletionReason", "transformation_label": "String Indexer"}], "feature": "TaskCompletionReason", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "COB Not Update-Released To Pt", "max": "Refiled Claim", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TaskCompletionReason"}, {"transformationsData": [{"feature_label": "TaskCompletionReasonCategory", "transformation_label": "String Indexer"}], "feature": "TaskCompletionReasonCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "COB Not Update-Released To Pt", "max": "Refiled Claim", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TaskCompletionReasonCateg..."}, {"transformationsData": [{"feature_label": "TaskType", "transformation_label": "String Indexer"}], "feature": "TaskType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "90 Days No Response", "max": "Waystar Eligibility Error", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TaskType"}, {"transformationsData": [{"feature_label": "TaskTypeCategory", "transformation_label": "String Indexer"}], "feature": "TaskTypeCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "AR", "max": "Unknown", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TaskTypeCategory"}, {"transformationsData": [{"feature_label": "TaskTypeGroup", "transformation_label": "String Indexer"}], "feature": "TaskTypeGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "RCM", "max": "Unknown", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TaskTypeGroup"}, {"transformationsData": [{"feature_label": "TaskStatus", "transformation_label": "String Indexer"}], "feature": "TaskStatus", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "Completed", "max": "Completed", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TaskStatus"}, {"transformationsData": [{"feature_label": "TaskStatusCategory", "transformation_label": "String Indexer"}], "feature": "TaskStatusCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "", "stddev": "", "min": "Completed", "max": "Completed", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TaskStatusCategory"}, {"transformationsData": [{"feature_label": "label_resolution", "transformation_label": "String Indexer"}], "feature": "label_resolution", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3984", "mean": "11.47", "stddev": "8.2", "min": "0", "max": "5", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "label_resolution"}, {"feature": "CPTCode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "9.4", "stddev": "18.32", "min": "0.0", "max": "131.0", "missing": "0"}, "updatedLabel": "CPTCode_stringindexer_tra..."}, {"feature": "CptModifier_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "1.96", "stddev": "3.77", "min": "0.0", "max": "34.0", "missing": "0"}, "updatedLabel": "CptModifier_stringindexer..."}, {"feature": "CptModifier2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "0.39", "stddev": "1.32", "min": "0.0", "max": "17.0", "missing": "0"}, "updatedLabel": "CptModifier2_stringindexe..."}, {"feature": "pdiagnosis1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "88.76", "stddev": "119.32", "min": "0.0", "max": "559.0", "missing": "0"}, "updatedLabel": "pdiagnosis1_stringindexer..."}, {"feature": "pdiagnosis2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "98.42", "stddev": "124.7", "min": "0.0", "max": "570.0", "missing": "0"}, "updatedLabel": "pdiagnosis2_stringindexer..."}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3984", "mean": "0.42", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "DepartmentCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "1.05", "stddev": "1.37", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "DepartmentCategory_string..."}, {"feature": "SpecialityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "2.28", "stddev": "2.59", "min": "0.0", "max": "13.0", "missing": "0"}, "updatedLabel": "SpecialityCategory_string..."}, {"feature": "PrimaryPayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "6.33", "stddev": "6.12", "min": "0.0", "max": "31.0", "missing": "0"}, "updatedLabel": "PrimaryPayerCategory_stri..."}, {"feature": "PrimaryPayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "4.83", "stddev": "4.64", "min": "0.0", "max": "23.0", "missing": "0"}, "updatedLabel": "PrimaryPayerGroup_stringi..."}, {"feature": "PrimaryPayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "22.31", "stddev": "33.93", "min": "0.0", "max": "210.0", "missing": "0"}, "updatedLabel": "PrimaryPayerName_stringin..."}, {"feature": "PrimaryPayerSubGroup1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3984", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PrimaryPayerSubGroup1_str..."}, {"feature": "PrimaryPayerSubGroup2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3984", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PrimaryPayerSubGroup2_str..."}, {"feature": "SecondPayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "3.45", "stddev": "4.8", "min": "0.0", "max": "30.0", "missing": "0"}, "updatedLabel": "SecondPayerCategory_strin..."}, {"feature": "SecondPayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "2.64", "stddev": "3.56", "min": "0.0", "max": "22.0", "missing": "0"}, "updatedLabel": "SecondPayerGroup_stringin..."}, {"feature": "SecondPayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "10.85", "stddev": "22.19", "min": "0.0", "max": "154.0", "missing": "0"}, "updatedLabel": "SecondPayerName_stringind..."}, {"feature": "SecondPayerSubGroup1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3984", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "SecondPayerSubGroup1_stri..."}, {"feature": "SecondPayerSubGroup2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3984", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "SecondPayerSubGroup2_stri..."}, {"feature": "TeritoryPayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "0.18", "stddev": "1.2", "min": "0.0", "max": "18.0", "missing": "0"}, "updatedLabel": "TeritoryPayerCategory_str..."}, {"feature": "TeritoryPayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "0.14", "stddev": "0.92", "min": "0.0", "max": "14.0", "missing": "0"}, "updatedLabel": "TeritoryPayerGroup_string..."}, {"feature": "TeritoryPayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "0.42", "stddev": "2.78", "min": "0.0", "max": "40.0", "missing": "0"}, "updatedLabel": "TeritoryPayerName_stringi..."}, {"feature": "TeritoryPayerSubGroup1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3984", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "TeritoryPayerSubGroup1_st..."}, {"feature": "TeritoryPayerSubGroup2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3984", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "TeritoryPayerSubGroup2_st..."}, {"feature": "ServiceItemName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "10.04", "stddev": "19.33", "min": "0.0", "max": "137.0", "missing": "0"}, "updatedLabel": "ServiceItemName_stringind..."}, {"feature": "ServiceItemCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "12.7", "stddev": "23.1", "min": "0.0", "max": "159.0", "missing": "0"}, "updatedLabel": "ServiceItemCategory_strin..."}, {"feature": "ServiceLocation_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3984", "mean": "0.13", "stddev": "0.41", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "ServiceLocation_stringind..."}, {"feature": "Reasoncode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "18.9", "stddev": "29.43", "min": "0.0", "max": "184.0", "missing": "0"}, "updatedLabel": "Reasoncode_stringindexer_..."}, {"feature": "ReasonCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "2.93", "stddev": "3.2", "min": "0.0", "max": "15.0", "missing": "0"}, "updatedLabel": "ReasonCategory_stringinde..."}, {"feature": "ReasonDescription_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "18.9", "stddev": "29.43", "min": "0.0", "max": "184.0", "missing": "0"}, "updatedLabel": "ReasonDescription_stringi..."}, {"feature": "TaskCompletedDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "3984", "mean": "15.83", "stddev": "8.66", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "TaskCompletedDate_dayofmo..."}, {"feature": "TaskCompletedDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "3984", "mean": "6.95", "stddev": "3.45", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "TaskCompletedDate_month"}, {"feature": "TaskCompletedDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "3984", "mean": "2019.0", "stddev": "1.02", "min": "2015", "max": "2021", "missing": "0"}, "updatedLabel": "TaskCompletedDate_year"}, {"feature": "TaskCreatedDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "3984", "mean": "16.05", "stddev": "8.63", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "TaskCreatedDate_dayofmont..."}, {"feature": "TaskCreatedDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "3984", "mean": "6.81", "stddev": "3.42", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "TaskCreatedDate_month"}, {"feature": "TaskCreatedDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "3984", "mean": "2018.99", "stddev": "1.02", "min": "2015", "max": "2021", "missing": "0"}, "updatedLabel": "TaskCreatedDate_year"}, {"feature": "TaskCompletionReason_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "1.09", "stddev": "1.52", "min": "0.0", "max": "13.0", "missing": "0"}, "updatedLabel": "TaskCompletionReason_stri..."}, {"feature": "TaskCompletionReasonCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "1.09", "stddev": "1.52", "min": "0.0", "max": "13.0", "missing": "0"}, "updatedLabel": "TaskCompletionReasonCateg..."}, {"feature": "TaskType_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "13.52", "stddev": "13.11", "min": "0.0", "max": "77.0", "missing": "0"}, "updatedLabel": "TaskType_stringindexer_tr..."}, {"feature": "TaskTypeCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "0.97", "stddev": "1.03", "min": "0.0", "max": "4.0", "missing": "0"}, "updatedLabel": "TaskTypeCategory_stringin..."}, {"feature": "TaskTypeGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3984", "mean": "0.42", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "TaskTypeGroup_stringindex..."}, {"feature": "TaskStatus_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3984", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "TaskStatus_stringindexer_..."}, {"feature": "TaskStatusCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3984", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "TaskStatusCategory_string..."}, {"feature": "label_resolution_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3984", "mean": "0.91", "stddev": "1.04", "min": "0.0", "max": "6.0", "missing": "0"}, "updatedLabel": "label_resolution_stringin..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run eligdenresolutionrecommendationautomlHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["CPTCode_stringindexer", "CptModifier_stringindexer", "CptModifier2_stringindexer", "pdiagnosis1_stringindexer", "pdiagnosis2_stringindexer", "Age", "Gender_stringindexer", "Amount", "DepartmentCategory_stringindexer", "SpecialityCategory_stringindexer", "PrimaryPayerCategory_stringindexer", "PrimaryPayerGroup_stringindexer", "PrimaryPayerName_stringindexer", "PrimaryPayerSubGroup1_stringindexer", "PrimaryPayerSubGroup2_stringindexer", "SecondPayerCategory_stringindexer", "SecondPayerGroup_stringindexer", "SecondPayerName_stringindexer", "SecondPayerSubGroup1_stringindexer", "SecondPayerSubGroup2_stringindexer", "TeritoryPayerCategory_stringindexer", "TeritoryPayerGroup_stringindexer", "TeritoryPayerName_stringindexer", "TeritoryPayerSubGroup1_stringindexer", "TeritoryPayerSubGroup2_stringindexer", "ServiceItemName_stringindexer", "ServiceItemCategory_stringindexer", "ServiceLocation_stringindexer", "Reasoncode_stringindexer", "ReasonCategory_stringindexer", "ReasonDescription_stringindexer", "TaskCompletionReasonID", "TaskCompletionReason_stringindexer", "TaskCompletionReasonCategory_stringindexer", "TaskType_stringindexer", "TaskTypeCategory_stringindexer", "TaskTypeGroup_stringindexer", "TaskStatus_stringindexer", "TaskStatusCategory_stringindexer", "TaskCompletedDate_dayofmonth", "TaskCompletedDate_month", "TaskCompletedDate_year", "TaskCreatedDate_dayofmonth", "TaskCreatedDate_month", "TaskCreatedDate_year"], "label_resolution_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

